In [1]:
import json
import requests
import re
from datetime import datetime
import time

In [2]:
BASE_URL = 'https://api.opendota.com/api/'
TEAMS = 'teams'
HEROES = 'heroes'
PROPLAYERS = 'proPlayers'
PROMATCHES = 'proMatches'
MATCHES = 'matches'
PLAYERS = 'players'
HEROES_STATS = 'heroStats'

In [3]:
def get_all_teams_from_opendota():
    
    response = requests.get(BASE_URL + TEAMS)

    try:
        team_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return team_data_json
    
    return 0

Get all the teams from OpenDota

In [4]:
all_teams_json = get_all_teams_from_opendota()

A sample showing the first value in the list. The items in the list is in a dictionary format and will have keys such as team_id, rating, wins, losses, etc....

In [5]:
all_teams_json[0]

{'team_id': 15,
 'rating': 1603.12,
 'wins': 1554,
 'losses': 937,
 'last_match_time': 1660520088,
 'name': 'PSG.LGD',
 'tag': 'PSG.LGD',
 'logo_url': 'https://steamcdn-a.akamaihd.net/apps/dota2/images/team_logos/15.png'}

The function search_item can look for a given string within a list of dictionaries

In [6]:
def search_regex(search_string, name_from_list):
    try:
        x = re.search(search_string, name_from_list, re.IGNORECASE).group()
    except Exception:
        return False
    return x

def search_item(item_to_search, item_key, data_source):
    item_filter =  filter(lambda y: search_regex(item_to_search, y[item_key]), data_source)
    return [item_info for item_info in item_filter]

A sample below searches for the word "Fnatic" from "name" key within the all_teams_json collection. It returned 2 results, manually select the right one

In [7]:
search_item('polaris', 'name', all_teams_json)

[{'team_id': 8525778,
  'rating': 1321.12,
  'wins': 191,
  'losses': 126,
  'last_match_time': 1663505523,
  'name': 'Polaris Esports',
  'tag': 'Polaris',
  'logo_url': 'https://steamusercontent-a.akamaihd.net/ugc/1831290045925630948/400C4E5FD779D964D9BEC5E44D414073ADC03790/'}]

In [8]:
def get_team_stat(team_id):
    response = requests.get(BASE_URL + TEAMS + '/' + str(team_id))

    try:
        team_stat_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return team_stat_json
    
    return 0

In [9]:
def get_match_info(match_id):
    response = requests.get(BASE_URL + MATCHES + '/' + str(match_id))

    try:
        match_info_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
        
    else:
        return match_info_json
    
    return 0

In [10]:
def get_player_info(account_id):
    response = requests.get(BASE_URL + PLAYERS + '/' + str(account_id))

    try:
        player_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return player_data_json
    
    return 0

In [11]:
def get_team_matches(team_id):
    
    response = requests.get(BASE_URL + TEAMS + '/' + str(team_id) + '/' + MATCHES)

    try:
        team_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return team_data_json
    
    return 0

Get all the team's matches using their team_id

In [19]:
team_matches_json = get_team_matches(8525778)

This shows the total number of matches for that team

In [13]:
len(team_matches_json)

317

Here's a sample of a match data 

In [14]:
team_matches_json[0]

{'match_id': 6764526982,
 'radiant_win': False,
 'radiant': False,
 'duration': 2807,
 'start_time': 1663505523,
 'leagueid': 14469,
 'league_name': 'Predator League 2022 Philippine Qualifiers',
 'cluster': 151,
 'opposing_team_id': 8597594,
 'opposing_team_name': 'GrindSky Esports',
 'opposing_team_logo': 'https://steamusercontent-a.akamaihd.net/ugc/1820012456995413028/340E1875F6D42B587AE030D8084A07785785A3E1/'}

Take note that 'start_time' is in Unix (Epoch) time format. It can be converted to standard format by using the 'datetime' library

In [27]:
def oneminutecountdown():
    for i in range(61, -1, -1):
        print('Exceeded max API calls, please wait:', i, 'seconds                                                ', end = '\r')
        time.sleep(1)

def combine_team_matches_data(team_matches_json, predict_team_id):  

    count = 1

    matches_total = len(team_matches_json)

    # 1 x API CALL
    predict_team_stat = get_team_stat(predict_team_id)

    for i in range(60, -1, -1):
        print('Exceeded max API calls, please wait:', i, 'seconds                                                ', end = '\r')
        time.sleep(1)
    
    for index, value in enumerate(team_matches_json):

        match_info = team_matches_json[index]
        
         
        if (count >= 60):
            oneminutecountdown()
            count = 1
        
        # 1 x API CALL
        match_info_detailed = get_match_info(match_info['match_id'])
        count += 1

        if (count >= 55):
            oneminutecountdown()
            count = 1
        
        # 5 x API CALL (5 players)
        predict_players = [{'account_id' : player['account_id'], 
                            'name' : player['name'],
                            'player_stats' : get_player_info(player['account_id'])
                            } for player in match_info_detailed['players'] if player['isRadiant'] == match_info['radiant']]
        count += 5
        
        if (count >= 55):
            oneminutecountdown()
            count = 1
            
        # 5 x API CALL (5 players)
        opposing_players = [{'account_id' : player['account_id'], 
                                'name' : player['name'],
                                'player_stats' : get_player_info(player['account_id'])
                            } for player in match_info_detailed['players'] if player['isRadiant'] != match_info['radiant']]
        count += 5

        team_matches_json[index]['predict_players'] = predict_players
        team_matches_json[index]['opposing_players'] = opposing_players

        if (count >= 60):
            oneminutecountdown()
            count = 1
        
        # 1 x API CALL
        opposing_team_stat = get_team_stat(match_info['opposing_team_id'])
        count += 1
    
        opposing_team = [{'opposing_team_wins_total' : opposing_team_stat['wins'], 
                            'opposing_team_losses_total' : opposing_team_stat['losses'], 
                            'opposing_team_rating' : opposing_team_stat['rating']}]
        
        predict_team = [{'predict_team_wins_total' : predict_team_stat['wins'], 
                            'predict_team_losses_total' : predict_team_stat['losses'], 
                            'predict_team_rating' : predict_team_stat['rating'],
                            'predict_team_name' : predict_team_stat['name'], 
                            'predict_team_id' : predict_team_stat['team_id']}]
        
        team_matches_json[index]['opposing_team_stat'] = opposing_team
        team_matches_json[index]['predict_team_stat'] = predict_team

        print('match number:', index, 'downloaded out of', matches_total, ':', matches_total - index, 'matches left to go', round((index / matches_total) * 100, 2),'% complete', end = '\r')
        

In [28]:
combine_team_matches_data(team_matches_json, 8525778)

In [29]:
# Save the JSON as a file in the working directory
def save_teams_data_to_file(team_name, team_matches_json):
    
    with open(team_name + '_data.json', 'w', encoding='utf-8') as f:
        json.dump(team_matches_json, f, ensure_ascii=False, indent=4)

In [30]:
len(team_matches_json[4])

15

In [31]:
for index, item in enumerate(team_matches_json):
    if (len(item) < 15):
        print(index)
    

In [32]:
team_matches_json[0]

{'match_id': 6764526982,
 'radiant_win': False,
 'radiant': False,
 'duration': 2807,
 'start_time': 1663505523,
 'leagueid': 14469,
 'league_name': 'Predator League 2022 Philippine Qualifiers',
 'cluster': 151,
 'opposing_team_id': 8597594,
 'opposing_team_name': 'GrindSky Esports',
 'opposing_team_logo': 'https://steamusercontent-a.akamaihd.net/ugc/1820012456995413028/340E1875F6D42B587AE030D8084A07785785A3E1/',
 'predict_players': [{'account_id': 146711951,
   'name': 'Force',
   'player_stats': {'solo_competitive_rank': 9329,
    'leaderboard_rank': 24,
    'profile': {'account_id': 146711951,
     'personaname': 'Force',
     'name': 'Force',
     'plus': True,
     'cheese': 0,
     'steamid': '76561198106977679',
     'avatar': 'https://avatars.akamai.steamstatic.com/430ecc52800a77f7ddc52921391b4215aeef9284.jpg',
     'avatarmedium': 'https://avatars.akamai.steamstatic.com/430ecc52800a77f7ddc52921391b4215aeef9284_medium.jpg',
     'avatarfull': 'https://avatars.akamai.steamstatic